# SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных.

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Задания.

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Загрузка данных и изучение общей информации.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'], 
                                                         db_config['host'], 
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
def request(sql):
    return pd.io.sql.read_sql(sql, con = engine)

## Смотрим на таблицы.

### Таблица books.

In [4]:
books = '''
          SELECT *
          FROM books
          LIMIT 5 
            '''
request(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Таблица authors.

In [5]:
authors = '''
          SELECT *
          FROM authors
          LIMIT 5 
            '''
request(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Таблица publishers.

In [6]:
publishers = '''
          SELECT *
          FROM publishers
          LIMIT 5 
            '''
request(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Таблица ratings.

In [7]:
ratings = '''
          SELECT *
          FROM ratings
          LIMIT 5 
            '''
request(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Таблица reviews.

In [8]:
reviews = '''
          SELECT *
          FROM reviews
          LIMIT 5 
            '''
request(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Подсчитаем количество строк.

In [9]:
pd.io.sql.read_sql('''
SELECT *,
       COUNT(*) OVER() AS table_size
FROM reviews 
LIMIT 1        
'''
, con = engine)

,review_id,book_id,username,text,table_size
0,1,1,brandtandrea,Mention society tell send professor analysis. ...,2793


## Задания.

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [10]:
'''SELECT COUNT(book_id)
FROM BOOKS
WHERE publication_date > 2000-01-01'''

'SELECT COUNT(book_id)\nFROM BOOKS\nWHERE publication_date > 2000-01-01'

In [11]:
book_count = '''
             SELECT COUNT(book_id)
             FROM books
             WHERE publication_date > '2000-01-01'
             '''
request(book_count)

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитайте количество обзоров и среднюю оценку.

In [12]:
count = '''
        SELECT b.book_id,
               title,
               COUNT(DISTINCT r.review_id) AS review,
               AVG(rt.rating) AS rating
        FROM books b
        LEFT JOIN reviews AS r ON b.book_id = r.book_id
        LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
        GROUP BY b.book_id
        ORDER BY COUNT(r.review_id) DESC
        '''
request(count)

,book_id,title,review,rating
0,948,Twilight (Twilight #1),7,3.662500
1,750,The Hobbit or There and Back Again,6,4.125000
2,673,The Catcher in the Rye,6,3.825581
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
...,...,...,...,...
995,221,Essential Tales and Poems,0,4.000000
996,387,Leonardo's Notebooks,0,4.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


Самая максимальное количество обзоров за книгу 7. Есть книги и без обзоров.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

In [13]:
publish = '''
          SELECT p.publisher,
                 COUNT(b.book_id) AS count             
          FROM books b
          LEFT JOIN publishers p ON b.publisher_id=p.publisher_id
          WHERE b.num_pages > 50
          GROUP BY p.publisher
          ORDER BY count DESC
          LIMIT 5
        '''
request(publish)

,publisher,count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Больше всего книг, с толщиной более 50 страниц выпустило издательство "Penguin Books" - 42. Затем идет издательство "Vintage" - 31. Меньше всего книг, с толщиной более 50 страниц выпустило издательство "Bantam" - 19.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [14]:
author = '''
        SELECT a.author,
               AVG(r.rating) AS rating
        FROM books AS b
        INNER JOIN (SELECT book_id,
                   AVG(rating) AS rating
                   FROM ratings
                   GROUP BY book_id
                   HAVING COUNT(rating_id) >= 50) AS r ON r.book_id=b.book_id
        LEFT JOIN authors AS a ON b.author_id=a.author_id
        GROUP BY a.author
        ORDER BY AVG(r.rating) DESC
            '''
request(author)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Авторами с самой высокой средней оценкой книг (с учетом с 50 или более оценками) являются J.K. Rowling и Mary GrandPré.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [15]:
user = '''
        SELECT COUNT(rv.review_id) / COUNT(DISTINCT rv.username) AS avg_review
        FROM reviews AS rv
        INNER JOIN (SELECT username,
                    COUNT(rating_id)
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating_id) > 50) AS r ON r.username=rv.username
'''
request(user)

,avg_review
0,24


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок составляет 24. Получается только половина оцененных книг обозреваются пользователями.

## Выводы.

- После 1 января 2000 года вышло 819 книг.
- Самая максимальное количество обзоров за книгу 7. Есть книги и без обзоров.
- Больше всего книг, с толщиной более 50 страниц выпустило издательство "Penguin Books" - 42. Затем идет издательство "Vintage" - 31. Меньше всего книг, с толщиной более 50 страниц выпустило издательство "Bantam" - 19.
- Авторами с самой высокой средней оценкой книг (с учетом с 50 или более оценками) являются J.K. Rowling и Mary GrandPré.
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок составляет 24. Получается только половина оцененных книг обозреваются пользователями.